# 🗄️ LangGraph + Apify + Supabase 통합 튜토리얼

크롤링한 Instagram Reel 데이터를 Supabase에 저장하고 관리하는 완전한 파이프라인을 구축합니다.

## 🎯 최종 목표
```
User: "natgeo 릴스 분석하고 저장해줘"
        ↓
LangGraph Agent
        ↓
┌─────────────────────────────────────┐
│  1. Apify: Instagram 크롤링         │
│  2. Supabase: 데이터 저장           │
│  3. LLM: 분석 및 인사이트 생성      │
└─────────────────────────────────────┘
        ↓
분석 결과 + DB 저장 완료!
```

## 📋 학습 내용
1. Supabase 프로젝트 설정 및 테이블 생성
2. 크롤링 데이터 → Supabase 저장
3. Supabase를 LangGraph Tool로 통합
4. 전체 파이프라인 구축
5. 데이터 조회 및 분석

---

## 0️⃣ 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install -q supabase langgraph langchain-apify langchain-openai pandas

In [2]:
import os

# API 키 설정
os.environ["APIFY_API_TOKEN"] = "<YOUR_APIFY_TOKEN>"      # https://console.apify.com/settings/integrations
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"        # https://platform.openai.com/api-keys

# Supabase 설정 (https://supabase.com/dashboard/project/_/settings/api)
SUPABASE_URL = os.environ["SUPABASE_URL"]  # Project URL
SUPABASE_KEY = os.environ["SUPABASE_KEY"]  # anon/public key

print("✅ 환경 변수 설정 완료!")

✅ 환경 변수 설정 완료!


---

## 1️⃣ Supabase 테이블 설계

### 1.1 테이블 구조 (SQL)

Supabase Dashboard > SQL Editor에서 아래 SQL을 실행하세요.

In [ ]:
# Supabase에서 실행할 SQL 스키마
create_tables_sql = """
-- =============================================
-- 1. Instagram 계정 테이블
-- =============================================
CREATE TABLE IF NOT EXISTS instagram_accounts (
    id BIGSERIAL PRIMARY KEY,
    username VARCHAR(255) UNIQUE NOT NULL,
    full_name VARCHAR(255),
    owner_id VARCHAR(255),
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- =============================================
-- 2. Instagram 릴스 테이블
-- =============================================
CREATE TABLE IF NOT EXISTS instagram_reels (
    id BIGSERIAL PRIMARY KEY,
    reel_id VARCHAR(255) UNIQUE NOT NULL,        -- Instagram 내부 ID
    short_code VARCHAR(255) UNIQUE,              -- 릴스 단축 코드
    account_id BIGINT REFERENCES instagram_accounts(id),
    
    -- 기본 정보
    url TEXT,
    caption TEXT,
    
    -- 통계
    likes_count INTEGER DEFAULT 0,
    comments_count INTEGER DEFAULT 0,
    views_count INTEGER DEFAULT 0,
    plays_count INTEGER DEFAULT 0,
    shares_count INTEGER DEFAULT 0,
    
    -- 영상 정보
    video_duration DECIMAL(10, 3),               -- 초 단위
    video_url TEXT,
    thumbnail_url TEXT,
    
    -- 추가 정보
    hashtags JSONB DEFAULT '[]'::JSONB,          -- 해시태그 배열
    mentions JSONB DEFAULT '[]'::JSONB,          -- 멘션 배열
    transcript TEXT,                              -- 자막/스크립트
    music_info JSONB,                            -- 음악 정보
    
    -- 메타데이터
    posted_at TIMESTAMPTZ,                       -- 게시 시간
    scraped_at TIMESTAMPTZ DEFAULT NOW(),        -- 크롤링 시간
    raw_data JSONB,                              -- 원본 데이터 전체
    
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- =============================================
-- 3. 릴스 댓글 테이블
-- =============================================
CREATE TABLE IF NOT EXISTS instagram_comments (
    id BIGSERIAL PRIMARY KEY,
    comment_id VARCHAR(255) UNIQUE NOT NULL,
    reel_id BIGINT REFERENCES instagram_reels(id),
    
    text TEXT,
    owner_username VARCHAR(255),
    likes_count INTEGER DEFAULT 0,
    replies_count INTEGER DEFAULT 0,
    
    commented_at TIMESTAMPTZ,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- =============================================
-- 4. 분석 결과 테이블
-- =============================================
CREATE TABLE IF NOT EXISTS analysis_results (
    id BIGSERIAL PRIMARY KEY,
    account_id BIGINT REFERENCES instagram_accounts(id),
    
    analysis_type VARCHAR(50),                   -- 'engagement', 'content', 'hashtag' 등
    result JSONB,                                -- 분석 결과 (유연한 구조)
    summary TEXT,                                -- LLM 생성 요약
    
    analyzed_at TIMESTAMPTZ DEFAULT NOW()
);

-- =============================================
-- 인덱스 생성 (성능 최적화)
-- =============================================
CREATE INDEX IF NOT EXISTS idx_reels_account ON instagram_reels(account_id);
CREATE INDEX IF NOT EXISTS idx_reels_posted_at ON instagram_reels(posted_at DESC);
CREATE INDEX IF NOT EXISTS idx_comments_reel ON instagram_comments(reel_id);
CREATE INDEX IF NOT EXISTS idx_reels_hashtags ON instagram_reels USING GIN(hashtags);

-- =============================================
-- updated_at 자동 업데이트 트리거
-- =============================================
CREATE OR REPLACE FUNCTION update_updated_at()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trigger_accounts_updated
    BEFORE UPDATE ON instagram_accounts
    FOR EACH ROW EXECUTE FUNCTION update_updated_at();

CREATE OR REPLACE TRIGGER trigger_reels_updated
    BEFORE UPDATE ON instagram_reels
    FOR EACH ROW EXECUTE FUNCTION update_updated_at();
"""

print("📋 위 SQL을 Supabase Dashboard > SQL Editor에서 실행하세요!")
print("\n" + "=" * 60)
print(create_tables_sql)

### 1.2 Supabase 클라이언트 초기화

In [3]:
from supabase import create_client, Client

# Supabase 클라이언트 생성
supabase: Client = create_client(
    os.environ["SUPABASE_URL"],
    os.environ["SUPABASE_KEY"]
)

print("✅ Supabase 클라이언트 초기화 완료!")

✅ Supabase 클라이언트 초기화 완료!


In [4]:
# 연결 테스트
try:
    result = supabase.table("influencers").select("*").limit(1).execute()
    print("✅ Supabase 연결 성공!")
    print(f"   테이블 접근 확인: influencers")
except Exception as e:
    print(f"❌ 연결 실패: {e}")
    print("   → 테이블을 먼저 생성했는지 확인하세요!")

✅ Supabase 연결 성공!
   테이블 접근 확인: influencers


---

## 2️⃣ 크롤링 데이터 → Supabase 저장

### 2.1 데이터 변환 함수

In [ ]:
from datetime import datetime
from typing import Optional
import json

def transform_reel_data(raw_reel: dict) -> dict:
    """
    Apify 크롤링 결과를 Supabase 스키마에 맞게 변환
    
    Args:
        raw_reel: Apify Instagram Reel Scraper 결과
    
    Returns:
        Supabase instagram_reels 테이블에 맞는 dict
    """
    return {
        "reel_id": raw_reel.get("id"),
        "short_code": raw_reel.get("shortCode"),
        "url": raw_reel.get("url"),
        "caption": raw_reel.get("caption"),
        
        # 통계
        "likes_count": raw_reel.get("likesCount", 0),
        "comments_count": raw_reel.get("commentsCount", 0),
        "views_count": raw_reel.get("videoViewCount", 0),
        "plays_count": raw_reel.get("videoPlayCount", 0),
        "shares_count": raw_reel.get("sharesCount", 0),
        
        # 영상 정보
        "video_duration": raw_reel.get("videoDuration"),
        "video_url": raw_reel.get("videoUrl"),
        "thumbnail_url": raw_reel.get("displayUrl") or (raw_reel.get("images", [None])[0]),
        
        # JSON 필드
        "hashtags": raw_reel.get("hashtags", []),
        "mentions": raw_reel.get("mentions", []),
        "transcript": raw_reel.get("transcript"),
        "music_info": raw_reel.get("musicInfo"),
        
        # 타임스탬프
        "posted_at": raw_reel.get("timestamp"),
        "scraped_at": datetime.now().isoformat(),
        
        # 원본 데이터 보관
        "raw_data": raw_reel,
    }


def transform_comment_data(raw_comment: dict, reel_db_id: int) -> dict:
    """
    댓글 데이터 변환
    """
    return {
        "comment_id": raw_comment.get("id"),
        "reel_id": reel_db_id,
        "text": raw_comment.get("text"),
        "owner_username": raw_comment.get("ownerUsername"),
        "likes_count": raw_comment.get("likesCount", 0),
        "replies_count": raw_comment.get("repliesCount", 0),
        "commented_at": raw_comment.get("timestamp"),
    }

print("✅ 데이터 변환 함수 정의 완료!")

### 2.2 Supabase 저장 클래스

In [ ]:
class InstagramDataStore:
    """
    Instagram 크롤링 데이터를 Supabase에 저장하는 클래스
    """
    
    def __init__(self, supabase_client: Client):
        self.client = supabase_client
    
    def get_or_create_account(self, username: str, full_name: str = None, owner_id: str = None) -> int:
        """
        계정 조회 또는 생성, ID 반환
        """
        # 기존 계정 조회
        result = (
            self.client.table("instagram_accounts")
            .select("id")
            .eq("username", username)
            .execute()
        )
        
        if result.data:
            return result.data[0]["id"]
        
        # 새 계정 생성
        new_account = (
            self.client.table("instagram_accounts")
            .insert({
                "username": username,
                "full_name": full_name,
                "owner_id": owner_id,
            })
            .execute()
        )
        
        return new_account.data[0]["id"]
    
    def save_reel(self, reel_data: dict, account_id: int) -> dict:
        """
        릴스 데이터 저장 (upsert)
        """
        transformed = transform_reel_data(reel_data)
        transformed["account_id"] = account_id
        
        # Upsert: 있으면 업데이트, 없으면 생성
        result = (
            self.client.table("instagram_reels")
            .upsert(transformed, on_conflict="reel_id")
            .execute()
        )
        
        return result.data[0] if result.data else None
    
    def save_comments(self, comments: list, reel_db_id: int) -> int:
        """
        댓글 일괄 저장
        """
        if not comments:
            return 0
        
        transformed_comments = [
            transform_comment_data(c, reel_db_id) 
            for c in comments
        ]
        
        result = (
            self.client.table("instagram_comments")
            .upsert(transformed_comments, on_conflict="comment_id")
            .execute()
        )
        
        return len(result.data) if result.data else 0
    
    def save_crawl_results(self, reels: list) -> dict:
        """
        크롤링 결과 전체 저장
        
        Args:
            reels: Apify 크롤링 결과 리스트
        
        Returns:
            저장 결과 요약
        """
        stats = {
            "accounts_created": 0,
            "reels_saved": 0,
            "comments_saved": 0,
            "errors": []
        }
        
        # 계정별로 그룹화
        accounts_processed = set()
        
        for reel in reels:
            try:
                username = reel.get("ownerUsername")
                if not username:
                    continue
                
                # 계정 처리
                if username not in accounts_processed:
                    account_id = self.get_or_create_account(
                        username=username,
                        full_name=reel.get("ownerFullName"),
                        owner_id=reel.get("ownerId")
                    )
                    accounts_processed.add(username)
                    stats["accounts_created"] += 1
                else:
                    # 이미 처리된 계정의 ID 가져오기
                    result = (
                        self.client.table("instagram_accounts")
                        .select("id")
                        .eq("username", username)
                        .execute()
                    )
                    account_id = result.data[0]["id"]
                
                # 릴스 저장
                saved_reel = self.save_reel(reel, account_id)
                if saved_reel:
                    stats["reels_saved"] += 1
                    
                    # 댓글 저장
                    comments = reel.get("latestComments", [])
                    if comments:
                        comments_count = self.save_comments(comments, saved_reel["id"])
                        stats["comments_saved"] += comments_count
                        
            except Exception as e:
                stats["errors"].append({
                    "reel_id": reel.get("id"),
                    "error": str(e)
                })
        
        return stats
    
    def save_analysis(self, account_id: int, analysis_type: str, result: dict, summary: str) -> dict:
        """
        분석 결과 저장
        """
        response = (
            self.client.table("analysis_results")
            .insert({
                "account_id": account_id,
                "analysis_type": analysis_type,
                "result": result,
                "summary": summary,
            })
            .execute()
        )
        
        return response.data[0] if response.data else None


# 인스턴스 생성
data_store = InstagramDataStore(supabase)
print("✅ InstagramDataStore 클래스 정의 완료!")

### 2.3 테스트: Apify 크롤링 → Supabase 저장

In [ ]:
from apify_client import ApifyClient

def crawl_and_save(username: str, limit: int = 5) -> dict:
    """
    Instagram 릴스 크롤링 후 Supabase 저장
    """
    print(f"🔍 크롤링 시작: @{username}")
    print(f"   결과 수: {limit}개")
    
    # 1. Apify 크롤링
    client = ApifyClient(os.environ["APIFY_API_TOKEN"])
    
    run_input = {
        "username": [username],
        "resultsLimit": limit,
    }
    
    print("⏳ Apify Actor 실행 중...")
    run = client.actor("xMc5Ga1oCONPmWJIa").call(run_input=run_input)
    
    # 결과 가져오기
    reels = list(client.dataset(run["defaultDatasetId"]).iterate_items())
    print(f"✅ 크롤링 완료: {len(reels)}개 릴스")
    
    # 2. Supabase 저장
    print("💾 Supabase 저장 중...")
    save_result = data_store.save_crawl_results(reels)
    
    print(f"\n📊 저장 결과:")
    print(f"   - 계정: {save_result['accounts_created']}개")
    print(f"   - 릴스: {save_result['reels_saved']}개")
    print(f"   - 댓글: {save_result['comments_saved']}개")
    
    if save_result['errors']:
        print(f"   - 에러: {len(save_result['errors'])}개")
    
    return {
        "reels": reels,
        "save_result": save_result
    }

print("✅ crawl_and_save 함수 정의 완료!")
print("\n사용법: result = crawl_and_save('natgeo', limit=5)")

In [ ]:
# 실행! (주석 해제)
# result = crawl_and_save("natgeo", limit=5)

---

## 3️⃣ Supabase를 LangGraph Tool로 통합

Agent가 직접 DB에 저장/조회할 수 있도록 Tool을 만듭니다.

In [ ]:
from langchain_core.tools import tool
from typing import Optional
import json

@tool
def save_reels_to_db(reels_json: str) -> str:
    """
    크롤링한 Instagram 릴스 데이터를 데이터베이스에 저장합니다.
    
    Args:
        reels_json: 릴스 데이터 JSON 문자열 (리스트 형태)
    
    Returns:
        저장 결과 요약
    """
    try:
        reels = json.loads(reels_json)
        result = data_store.save_crawl_results(reels)
        
        return f"""
✅ 저장 완료!
- 계정: {result['accounts_created']}개
- 릴스: {result['reels_saved']}개  
- 댓글: {result['comments_saved']}개
- 에러: {len(result['errors'])}개
"""
    except Exception as e:
        return f"❌ 저장 실패: {str(e)}"


@tool
def query_reels_from_db(
    username: Optional[str] = None,
    limit: int = 10,
    order_by: str = "likes_count"
) -> str:
    """
    데이터베이스에서 저장된 릴스를 조회합니다.
    
    Args:
        username: 특정 계정 필터 (없으면 전체)
        limit: 조회할 개수
        order_by: 정렬 기준 (likes_count, views_count, posted_at)
    
    Returns:
        조회된 릴스 데이터
    """
    try:
        query = (
            supabase.table("instagram_reels")
            .select("""
                short_code, caption, likes_count, comments_count, 
                views_count, video_duration, posted_at,
                instagram_accounts(username)
            """)
            .order(order_by, desc=True)
            .limit(limit)
        )
        
        # 계정 필터
        if username:
            # 먼저 account_id 조회
            account = (
                supabase.table("instagram_accounts")
                .select("id")
                .eq("username", username)
                .execute()
            )
            if account.data:
                query = query.eq("account_id", account.data[0]["id"])
        
        result = query.execute()
        
        if not result.data:
            return "📭 저장된 릴스가 없습니다."
        
        # 결과 포맷팅
        output = f"📊 조회 결과: {len(result.data)}개 릴스\n\n"
        
        for i, reel in enumerate(result.data, 1):
            account_info = reel.get("instagram_accounts", {})
            output += f"""
{i}. @{account_info.get('username', 'unknown')} - {reel['short_code']}
   👍 {reel['likes_count']:,} | 💬 {reel['comments_count']:,} | 👁️ {reel['views_count']:,}
   📝 {(reel['caption'] or '')[:100]}...
"""
        
        return output
        
    except Exception as e:
        return f"❌ 조회 실패: {str(e)}"


@tool
def get_account_stats(username: str) -> str:
    """
    특정 계정의 릴스 통계를 조회합니다.
    
    Args:
        username: Instagram 사용자명
    
    Returns:
        계정 통계 요약
    """
    try:
        # 계정 조회
        account = (
            supabase.table("instagram_accounts")
            .select("id, username, full_name")
            .eq("username", username)
            .execute()
        )
        
        if not account.data:
            return f"❌ @{username} 계정을 찾을 수 없습니다."
        
        account_id = account.data[0]["id"]
        
        # 릴스 통계 조회
        reels = (
            supabase.table("instagram_reels")
            .select("likes_count, comments_count, views_count, video_duration")
            .eq("account_id", account_id)
            .execute()
        )
        
        if not reels.data:
            return f"📭 @{username}의 저장된 릴스가 없습니다."
        
        # 통계 계산
        total_reels = len(reels.data)
        total_likes = sum(r["likes_count"] or 0 for r in reels.data)
        total_comments = sum(r["comments_count"] or 0 for r in reels.data)
        total_views = sum(r["views_count"] or 0 for r in reels.data)
        avg_duration = sum(r["video_duration"] or 0 for r in reels.data) / total_reels
        
        avg_engagement = ((total_likes + total_comments) / total_views * 100) if total_views > 0 else 0
        
        return f"""
📊 @{username} 통계
{'=' * 40}
📹 총 릴스: {total_reels}개
❤️ 총 좋아요: {total_likes:,}
💬 총 댓글: {total_comments:,}
👁️ 총 조회수: {total_views:,}
⏱️ 평균 길이: {avg_duration:.1f}초

📈 평균 Engagement Rate: {avg_engagement:.2f}%
📊 평균 좋아요/릴스: {total_likes/total_reels:,.0f}
📊 평균 댓글/릴스: {total_comments/total_reels:,.0f}
"""
        
    except Exception as e:
        return f"❌ 통계 조회 실패: {str(e)}"


@tool
def analyze_hashtags_from_db(username: str) -> str:
    """
    저장된 릴스에서 해시태그 분석을 수행합니다.
    
    Args:
        username: Instagram 사용자명
    
    Returns:
        해시태그 분석 결과
    """
    try:
        # 계정의 릴스에서 해시태그 조회
        account = (
            supabase.table("instagram_accounts")
            .select("id")
            .eq("username", username)
            .execute()
        )
        
        if not account.data:
            return f"❌ @{username} 계정을 찾을 수 없습니다."
        
        reels = (
            supabase.table("instagram_reels")
            .select("hashtags")
            .eq("account_id", account.data[0]["id"])
            .execute()
        )
        
        # 해시태그 집계
        from collections import Counter
        all_hashtags = []
        for reel in reels.data:
            hashtags = reel.get("hashtags", [])
            if hashtags:
                all_hashtags.extend(hashtags)
        
        if not all_hashtags:
            return f"📭 @{username}의 해시태그가 없습니다."
        
        counter = Counter(all_hashtags)
        top_10 = counter.most_common(10)
        
        output = f"""
🏷️ @{username} 해시태그 분석
{'=' * 40}
총 사용: {len(all_hashtags)}개
고유 태그: {len(counter)}개

🔝 Top 10 해시태그:
"""
        for tag, count in top_10:
            output += f"  #{tag}: {count}회\n"
        
        return output
        
    except Exception as e:
        return f"❌ 해시태그 분석 실패: {str(e)}"


print("✅ Supabase Tools 정의 완료!")
print("\n🛠️ 사용 가능한 도구:")
print("  - save_reels_to_db: 릴스 데이터 저장")
print("  - query_reels_from_db: 릴스 조회")
print("  - get_account_stats: 계정 통계")
print("  - analyze_hashtags_from_db: 해시태그 분석")

---

## 4️⃣ 전체 파이프라인 Agent 구축

크롤링 → 저장 → 분석을 한 번에 처리하는 Agent

In [ ]:
from langchain_apify import ApifyActorsTool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

# 모든 도구 통합
all_tools = [
    # Apify 크롤링
    ApifyActorsTool("apify/instagram-reel-scraper"),
    
    # Supabase 저장/조회
    save_reels_to_db,
    query_reels_from_db,
    get_account_stats,
    analyze_hashtags_from_db,
]

# System Prompt
FULL_PIPELINE_PROMPT = """
당신은 Instagram 데이터 분석 전문 AI Agent입니다.

## 사용 가능한 도구

### 크롤링
- **instagram-reel-scraper**: Instagram 릴스 데이터 수집
  - username 파라미터로 계정 지정
  - resultsLimit로 개수 제한

### 데이터베이스
- **save_reels_to_db**: 크롤링 결과를 DB에 저장
- **query_reels_from_db**: 저장된 릴스 조회
- **get_account_stats**: 계정 통계 조회
- **analyze_hashtags_from_db**: 해시태그 분석

## 워크플로우

사용자가 분석을 요청하면:
1. 먼저 DB에 데이터가 있는지 확인 (query_reels_from_db)
2. 데이터가 없으면 크롤링 실행 (instagram-reel-scraper)
3. 크롤링 결과를 DB에 저장 (save_reels_to_db)
4. 저장된 데이터로 분석 수행
5. 인사이트와 개선 제안 제공

## 주의사항
- 크롤링 후에는 반드시 save_reels_to_db로 저장
- JSON 데이터는 문자열로 변환하여 전달
- 한국어로 답변
"""

# Agent 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)
full_agent = create_react_agent(
    llm.bind(system=FULL_PIPELINE_PROMPT),
    all_tools
)

print("✅ 전체 파이프라인 Agent 생성 완료!")
print(f"🛠️ 통합된 도구 수: {len(all_tools)}개")

In [ ]:
async def run_full_pipeline(query: str):
    """
    전체 파이프라인 Agent 실행
    """
    print(f"🔍 Query: {query}")
    print("=" * 60)
    
    async for event in full_agent.astream_events(
        {"messages": [HumanMessage(content=query)]},
        version="v2"
    ):
        if event["event"] == "on_tool_start":
            print(f"\n🔧 도구 호출: {event['name']}")
        elif event["event"] == "on_tool_end":
            print(f"   ✅ 완료")
        elif event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if hasattr(chunk, 'content') and chunk.content:
                print(chunk.content, end="", flush=True)
    
    print("\n" + "=" * 60)

print("✅ run_full_pipeline 함수 정의 완료!")

In [ ]:
# 실행 예시들 (주석 해제하여 실행)

# 예시 1: 크롤링 + 저장 + 분석
# await run_full_pipeline("natgeo 인스타그램 릴스 5개 크롤링해서 DB에 저장하고 분석해줘")

# 예시 2: DB에서 조회 + 분석
# await run_full_pipeline("저장된 natgeo 릴스 통계 보여줘")

# 예시 3: 해시태그 분석
# await run_full_pipeline("natgeo의 해시태그 전략 분석해줘")

# 예시 4: 인기 콘텐츠 조회
# await run_full_pipeline("좋아요가 가장 많은 릴스 Top 5 보여줘")

---

## 5️⃣ 데이터 조회 및 분석 유틸리티

### 5.1 Pandas로 데이터 분석

In [ ]:
import pandas as pd

def get_reels_dataframe(username: str = None) -> pd.DataFrame:
    """
    Supabase에서 릴스 데이터를 DataFrame으로 가져오기
    """
    query = (
        supabase.table("instagram_reels")
        .select("""
            short_code, caption, likes_count, comments_count, views_count,
            plays_count, shares_count, video_duration, hashtags, posted_at,
            instagram_accounts(username, full_name)
        """)
        .order("posted_at", desc=True)
    )
    
    if username:
        account = (
            supabase.table("instagram_accounts")
            .select("id")
            .eq("username", username)
            .execute()
        )
        if account.data:
            query = query.eq("account_id", account.data[0]["id"])
    
    result = query.execute()
    
    if not result.data:
        return pd.DataFrame()
    
    df = pd.DataFrame(result.data)
    
    # nested dict 처리
    if 'instagram_accounts' in df.columns:
        df['username'] = df['instagram_accounts'].apply(lambda x: x.get('username') if x else None)
        df = df.drop(columns=['instagram_accounts'])
    
    return df

print("✅ get_reels_dataframe 함수 정의 완료!")

In [ ]:
# 데이터 조회 및 분석 (주석 해제)
# df = get_reels_dataframe("natgeo")
# display(df.head())

# 기본 통계
# print(df[['likes_count', 'comments_count', 'views_count']].describe())

### 5.2 시각화

In [ ]:
import matplotlib.pyplot as plt

def visualize_account_performance(username: str):
    """
    계정 성과 시각화
    """
    df = get_reels_dataframe(username)
    
    if df.empty:
        print(f"❌ @{username}의 데이터가 없습니다.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. 좋아요 분포
    axes[0, 0].bar(range(len(df)), df['likes_count'], color='coral')
    axes[0, 0].set_title(f'@{username} - 릴스별 좋아요')
    axes[0, 0].set_xlabel('릴스')
    axes[0, 0].set_ylabel('좋아요 수')
    
    # 2. Engagement Rate
    df['engagement_rate'] = (df['likes_count'] + df['comments_count']) / df['views_count'].replace(0, 1) * 100
    axes[0, 1].bar(range(len(df)), df['engagement_rate'], color='steelblue')
    axes[0, 1].set_title('릴스별 Engagement Rate')
    axes[0, 1].set_xlabel('릴스')
    axes[0, 1].set_ylabel('Engagement Rate (%)')
    
    # 3. 영상 길이 분포
    if 'video_duration' in df.columns:
        axes[1, 0].hist(df['video_duration'].dropna(), bins=10, color='green', alpha=0.7)
        axes[1, 0].set_title('영상 길이 분포')
        axes[1, 0].set_xlabel('길이 (초)')
        axes[1, 0].set_ylabel('빈도')
    
    # 4. 조회수 vs 좋아요 (상관관계)
    axes[1, 1].scatter(df['views_count'], df['likes_count'], alpha=0.6)
    axes[1, 1].set_title('조회수 vs 좋아요')
    axes[1, 1].set_xlabel('조회수')
    axes[1, 1].set_ylabel('좋아요')
    
    plt.tight_layout()
    plt.show()

print("✅ visualize_account_performance 함수 정의 완료!")

In [ ]:
# 시각화 실행 (주석 해제)
# visualize_account_performance("natgeo")

---

## 6️⃣ 프로덕션 고려사항

### 6.1 환경 변수 관리

In [ ]:
# .env 파일 예시
env_template = """
# .env

# Apify
APIFY_API_TOKEN=apify_api_xxxxx

# OpenAI
OPENAI_API_KEY=sk-xxxxx

# Supabase
SUPABASE_URL=https://xxxxx.supabase.co
SUPABASE_KEY=eyJxxxxx  # anon key (public)
SUPABASE_SERVICE_KEY=eyJxxxxx  # service_role key (서버 전용, 선택)
"""

print("📋 .env 파일 템플릿:")
print(env_template)

### 6.2 Row Level Security (RLS)

In [ ]:
# RLS 정책 예시 (Supabase에서 실행)
rls_sql = """
-- RLS 활성화
ALTER TABLE instagram_reels ENABLE ROW LEVEL SECURITY;
ALTER TABLE instagram_accounts ENABLE ROW LEVEL SECURITY;

-- 읽기 정책: 모든 사용자가 조회 가능
CREATE POLICY "Public read access"
ON instagram_reels FOR SELECT
USING (true);

-- 쓰기 정책: Service Role만 가능
CREATE POLICY "Service role insert"
ON instagram_reels FOR INSERT
WITH CHECK (auth.role() = 'service_role');

-- 또는 인증된 사용자만
-- WITH CHECK (auth.uid() IS NOT NULL);
"""

print("🔒 RLS 정책 예시 (선택사항):")
print(rls_sql)

### 6.3 에러 처리 및 재시도

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10)
)
def robust_save_to_supabase(data: list) -> dict:
    """
    재시도 로직이 포함된 저장 함수
    """
    try:
        return data_store.save_crawl_results(data)
    except Exception as e:
        print(f"⚠️ 저장 실패, 재시도 중... ({e})")
        raise

print("✅ robust_save_to_supabase 함수 정의 완료!")

---

## 📚 요약

### 완성된 파이프라인

```
사용자 요청
    ↓
LangGraph Agent
    ├─ Apify Tool → Instagram 크롤링
    ├─ save_reels_to_db → Supabase 저장
    ├─ query_reels_from_db → 데이터 조회
    ├─ get_account_stats → 통계 분석
    └─ analyze_hashtags_from_db → 해시태그 분석
    ↓
분석 결과 + DB 영구 저장
```

### 테이블 구조

| 테이블 | 용도 |
|--------|------|
| instagram_accounts | 계정 정보 |
| instagram_reels | 릴스 데이터 |
| instagram_comments | 댓글 |
| analysis_results | 분석 결과 |

### 🔗 참고 자료
- [Supabase Python Docs](https://supabase.com/docs/reference/python/)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [Apify Docs](https://docs.apify.com/)

In [ ]:
print("🎉 튜토리얼 완료!")
print("\n이제 자연어로 요청하면 Agent가:")
print("  1. Instagram 릴스를 크롤링하고")
print("  2. Supabase에 저장하고")
print("  3. 분석 결과를 제공합니다!")